<a href="https://colab.research.google.com/github/toinnn/Torax_Xray-Health-Care/blob/main/Prepare_data_2_Yolov8__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from torchvision.io import decode_png, read_image , ImageReadMode
from PIL   import   Image
import polars as pl
import shutil
import torch
import json

path_project  = "/content/drive/Othercomputers/Meu laptop/Dataset_work_space/Torax_Xray"
image_dir_path = path_project + "/images/"
data_intro_path = path_project + "/Data_Entry_2017.csv"
label2class_path = path_project + "/target_dict_label2class.json"
BB_Original_path  = path_project + "/BBox_List_2017.csv"
BB_YoloFormat_path = path_project + "/Yolo_BBox_List_2017.csv"
BB_YoloFmt_agg_path = path_project + "/Yolo_BBox_agg_List_2017.csv"
BB_YoloFmt_test_path = path_project + "/Yolo_BBox_test_List_2017.csv"
BB_YoloFmt_train_path = path_project + "/Yolo_BBox_train_List_2017.csv"


def load_image_nvjpngl_gpu(image_path):
    """
    Loads an image from the specified file path using NVJPEG decoder on GPU and returns a PyTorch tensor.
    Args:
        image_path (str): The path to the image file.
    Returns:
        torch.Tensor: The PyTorch tensor representing the image.
    """
    data = read_image(image_path , mode = ImageReadMode.GRAY )
    return data.float()

    tensor = decode_png(data).float()
    return tensor

def BB_NIH_2_YOLO_Format(bb):
    x , y , w , h = bb
    x_new = x + w/2
    y_new = y + h/2

    return x_new , y_new , w , h

In [6]:
load_image_nvjpngl_gpu('/content/drive/Othercomputers/Meu laptop/Dataset_work_space/Torax_Xray/images/00005762_004.png')

tensor([[[  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         ...,
         [ 97., 163., 152.,  ...,  44.,  46.,  26.],
         [106., 179., 165.,  ...,  47.,  49.,  27.],
         [ 53.,  89.,  81.,  ...,  23.,  23.,  13.]]])

In [ ]:
# Data_Entry  = pl.read_csv(Data_Entry_path)
# Data_Entry = {row['Image Index'] : row['Finding Labels'].split("|") for row in Data_Entry[['Image Index' ,'Finding Labels' ]].iter_rows(named=True)}

BB_Yolo  = []
BB_Entry = pl.read_csv(BB_Original_path)

for row in BB_Entry[['Image Index' ,'Finding Label', "Bbox [x" , "y" , "w" , "h]" ]].iter_rows(named=True) :
    BB_Yolo += [ {'Image Index' : row['Image Index'] , 'Finding Label' : row['Finding Label'] , "Bounding Box" : "{},{},{},{}".format(*( BB_NIH_2_YOLO_Format(  (row["Bbox [x"]  , row["y"] , row["w"] , row["h]"])  ) )) } ]

BB_Yolo = pl.DataFrame(BB_Yolo)
BB_Yolo.write_csv( BB_YoloFormat_path )

In [ ]:
print(BB_Yolo)
print(BB_Yolo['Finding Label'].unique() , "\n\n\n" , BB_Yolo['Finding Label'].value_counts())
print("\n\n\n" , BB_Yolo['Image Index'].unique() )


shape: (984, 3)
┌──────────────────┬───────────────┬───────────────────────────────────┐
│ Image Index      ┆ Finding Label ┆ Bounding Box                      │
│ ---              ┆ ---           ┆ ---                               │
│ str              ┆ str           ┆ str                               │
╞══════════════════╪═══════════════╪═══════════════════════════════════╡
│ 00013118_008.png ┆ Atelectasis   ┆ 268.47457627118655,586.612437102… │
│ 00014716_007.png ┆ Atelectasis   ┆ 778.8474576271185,288.2892611228… │
│ 00029817_009.png ┆ Atelectasis   ┆ 299.38983050847446,425.527691340… │
│ 00014687_001.png ┆ Atelectasis   ┆ 796.745762711864,522.61243710275… │
│ …                ┆ …             ┆ …                                 │
│ 00025769_001.png ┆ Atelectasis   ┆ 753.607118055556,604.34963541666… │
│ 00016837_002.png ┆ Atelectasis   ┆ 276.87822916666653,706.180746527… │
│ 00020124_003.png ┆ Atelectasis   ┆ 297.358229166667,632.22519097222… │
│ 00026920_000.png ┆ Atelectasis   

In [ ]:
# [ {'Image Index' : row['Image Index'] , 'Finding Label' : row['Finding Label'] , "Bounding Box" : "{},{},{},{}".format(*( BB_NIH_2_YOLO_Format(  (row["Bbox [x"]  , row["y"] , row["w"] , row["h]"])  ) )) } ]
BB_Yolo_v2 = {}
for row in BB_Yolo[['Image Index' ,'Finding Label', "Bounding Box" ]].iter_rows(named=True) :
    try :
        BB_Yolo_v2[row['Image Index']] += [(row['Finding Label']  , row["Bounding Box"]  ) ]
    except :
        BB_Yolo_v2[row['Image Index']]  = [(row['Finding Label']  , row["Bounding Box"]  ) ]


In [ ]:
print(len(BB_Yolo_v2))
BB_Yolo_v3 = []
for key in BB_Yolo_v2 :
    BB_Yolo_v3 += [{'Image Index' : key , 'Finding Label' : "{}".format( [ i[0] for i in BB_Yolo_v2[key]] ), "Bounding Box" : "{}".format( [ i[1] for i in BB_Yolo_v2[key]] ) }]
BB_Yolo_v3 = pl.DataFrame(BB_Yolo_v3)


print(BB_Yolo_v3)
BB_Yolo_v3.write_csv( BB_YoloFmt_agg_path )

# for row in BB_Yolo_v3 :
# print(BB_Yolo_v3["Image Index"].to_list())

880
shape: (880, 3)
┌──────────────────┬─────────────────┬───────────────────────────────────┐
│ Image Index      ┆ Finding Label   ┆ Bounding Box                      │
│ ---              ┆ ---             ┆ ---                               │
│ str              ┆ str             ┆ str                               │
╞══════════════════╪═════════════════╪═══════════════════════════════════╡
│ 00013118_008.png ┆ ['Atelectasis'] ┆ ['268.47457627118655,586.6124371… │
│ 00014716_007.png ┆ ['Atelectasis'] ┆ ['778.8474576271185,288.28926112… │
│ 00029817_009.png ┆ ['Atelectasis'] ┆ ['299.38983050847446,425.5276913… │
│ 00014687_001.png ┆ ['Atelectasis'] ┆ ['796.745762711864,522.612437102… │
│ …                ┆ …               ┆ …                                 │
│ 00001320_003.png ┆ ['Atelectasis'] ┆ ['244.4515624999999,559.97630208… │
│ 00013187_002.png ┆ ['Atelectasis'] ┆ ['568.149340277778,655.549635416… │
│ 00025769_001.png ┆ ['Atelectasis'] ┆ ['753.607118055556,604.349635416… │
│ 000

In [ ]:
from sklearn.model_selection import train_test_split

images_ids = BB_Yolo_v3["Image Index"].to_list()
dados_treino, dados_teste = train_test_split( images_ids , test_size=0.2, random_state=42)

print(len(dados_teste))

# Dividindo o DataFrame com base na lista de IDs
df_teste  = BB_Yolo_v3.filter( pl.col("Image Index").is_in( dados_teste ) )
df_treino = BB_Yolo_v3.filter( pl.col("Image Index").is_in( dados_treino ) )

print(df_teste)
print(df_treino)

df_treino.write_csv( BB_YoloFmt_train_path )
df_teste.write_csv( BB_YoloFmt_test_path )


176
shape: (176, 3)
┌──────────────────┬──────────────────┬───────────────────────────────────┐
│ Image Index      ┆ Finding Label    ┆ Bounding Box                      │
│ ---              ┆ ---              ┆ ---                               │
│ str              ┆ str              ┆ str                               │
╞══════════════════╪══════════════════╪═══════════════════════════════════╡
│ 00003148_004.png ┆ ['Atelectasis']  ┆ ['624.2711864406774,595.81468485… │
│ 00030674_000.png ┆ ['Atelectasis']  ┆ ['746.847457627119,357.188708289… │
│ 00030635_001.png ┆ ['Atelectasis']  ┆ ['696.949152542373,601.798877780… │
│ 00019124_045.png ┆ ['Atelectasis']  ┆ ['395.389830508474,555.154809984… │
│ …                ┆ …                ┆ …                                 │
│ 00019706_002.png ┆ ['Pneumothorax'] ┆ ['652.867724867725,149.536507936… │
│ 00019124_006.png ┆ ['Pneumothorax'] ┆ ['700.5460317460314,211.84338624… │
│ 00012299_002.png ┆ ['Atelectasis']  ┆ ['524.9137847222225,615.7274

In [7]:
df_treino = pl.read_csv( BB_YoloFmt_train_path )
df_teste  = pl.read_csv( BB_YoloFmt_test_path )

print(df_teste)
print(df_treino)

shape: (176, 3)
┌──────────────────┬──────────────────┬───────────────────────────────────┐
│ Image Index      ┆ Finding Label    ┆ Bounding Box                      │
│ ---              ┆ ---              ┆ ---                               │
│ str              ┆ str              ┆ str                               │
╞══════════════════╪══════════════════╪═══════════════════════════════════╡
│ 00003148_004.png ┆ ['Atelectasis']  ┆ ['624.2711864406774,595.81468485… │
│ 00030674_000.png ┆ ['Atelectasis']  ┆ ['746.847457627119,357.188708289… │
│ 00030635_001.png ┆ ['Atelectasis']  ┆ ['696.949152542373,601.798877780… │
│ 00019124_045.png ┆ ['Atelectasis']  ┆ ['395.389830508474,555.154809984… │
│ …                ┆ …                ┆ …                                 │
│ 00019706_002.png ┆ ['Pneumothorax'] ┆ ['652.867724867725,149.536507936… │
│ 00019124_006.png ┆ ['Pneumothorax'] ┆ ['700.5460317460314,211.84338624… │
│ 00012299_002.png ┆ ['Atelectasis']  ┆ ['524.9137847222225,615.72741319

In [9]:
import ast
def BB_absolute_2_normalized(x_c , y_c , w , h , Image_w , Image_h):
    return x_c/Image_w  ,  y_c/Image_h  ,  w/Image_w  ,  h/Image_h
def str_2_float_list(lista):
    return [ float(i) for i in lista]
def df_2_dataset_Yolo_fmt(dframe , dir_base_path , labels) :
    for row in dframe[['Image Index' ,'Finding Label', "Bounding Box" ]].iter_rows(named=True) :
        # IMAGES :
        ID = row['Image Index']

        caminho_origem  = image_dir_path  + f"{ID}"
        caminho_destino = dir_base_path +  "images/"

        # shutil.copy(caminho_origem, caminho_destino)

        # LABELS :
        #### DATASET  :
        target = ""
        # print(type(row['Finding Label']))
        find_label = json.loads(row['Finding Label'].replace("'", "\""))
        Bounding_Box = [ str_2_float_list(  i.split(",")  )  for  i in ast.literal_eval(row['Bounding Box']) ] #json.loads(row["Bounding Box"].replace("'", "\""))
        for i in range(len(find_label)) :
            # print(Bounding_Box)
            # print(find_label)
            x_c , y_c , w , h = Bounding_Box[i]
            x_c , y_c , w , h = BB_absolute_2_normalized(x_c , y_c , w , h , 1024 , 1024)
            target += f"{labels[  find_label[i] ]} {x_c} {y_c} {w} {h}" + "\n"
        target = target[:-1]

        with open(dir_base_path +  f"labels/{ID[:-4]}.txt", 'w') as arquivo:
            arquivo.write(target)

yolo_teste_path = path_project + "/Yolo_Object_Detection/val/"
yolo_treino_path = path_project + "/Yolo_Object_Detection/train/"

# Copiar o arquivo para o Google Drive

labels_instance = {"Atelectasis":0 , "Cardiomegaly" : 1, "Pneumothorax" : 2 , "Effusion" : 3 , "Nodule" : 4 , "Infiltrate" : 5 , "Mass" : 6 , "Pneumonia" : 7}

df_2_dataset_Yolo_fmt(df_teste , yolo_teste_path  , labels_instance )
df_2_dataset_Yolo_fmt(df_treino, yolo_treino_path , labels_instance )


